<a href="https://colab.research.google.com/github/andreusjh99/Learning-Tensorflow2.0/blob/master/SequentialAPI(basics).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sequential API**

Author: [Chia Jing Heng](https://github.com/andreusjh99)

This notebook explores the basics of using keras.Sequential API to create and a model architecture. The bulk of this tutorial is from the [tensorflow official guide](https://www.tensorflow.org/guide/keras/sequential_model).

# *When to use a Sequential model?
A `sequential` model is appropriate for a **plain stack of layers** where each layer has exactly one input tensor and one output tensor.

It is not appropriate when:
* Your model or any of its layers has multiple inputs or multiple outputs
* You need to do layer sharing
* You want a non-linear topology (eg a residual connection, a multi-branch model etc.)

*Note: the layers, losses, metrics etc. mentioned in this tutorial will be explained briefly in the appendix. For more info: [layers](https://keras.io/api/layers/), [losses](https://keras.io/api/losses/), [metrics](https://keras.io/api/metrics/).*

## Setup


In [1]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.2.0


# Creating a Sequential model

Creating a Sequential model involves defining the layers, and this is helped by the Layers API.

## Specifying input shape in advance
Generally all layers in Keras need to know the shape of their inputs in order to be able to create their weights. 

The example below illustrates this.

In [3]:
layer = layers.Dense(3)
layer.weights # Empty

[]

In [4]:
x = tf.ones((1, 4))
y = layer(x)
layer.weights # Now it has weights of shape (4, 3) and bias (3, )

[<tf.Variable 'dense/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.20316261,  0.06685311, -0.5065591 ],
        [ 0.42300308, -0.24743915, -0.01107436],
        [-0.84601885, -0.33679044,  0.07068479],
        [ 0.76290965,  0.5092368 , -0.1899758 ]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

As such, it is usual practice to instantiate a Sequential model with an input shape, as will be shown below.

## Sequential Model
There are generally 2 ways of creating a Sequential model:
1. passing a list of layers to the Sequential constructor
2. create a Sequential model incrementally by using the `add()` method.

Note: both ways give the same result!

### 1. passing a list of layers to the Sequential constructor

> This method does not allow you to specify the input shape as you are instantiating the model.

> Only after calling the model on an input that you would have specified the input shape for the model. This is not ideal as you would want to be able to inspect the architecture of the model before using it.

In [6]:
# 1. Passing a list of layers to the Sequential constructor
model1 = keras.Sequential(
    [
        layers.Dense(2, activation='relu', name="input_layer"),
        layers.Dense(3, activation='relu', name="hidden_dense_1"),
        layers.Dense(4, name="output_layer")
    ], name = "model_1", 
)

### 2. create a Sequential model incrementally by using the `add()` method
> This method allows you to add an `keras.Input` object specifying the input shape before defining the layers. As such this is the more widely used way of creating a model.

In [8]:
# 2. Incrementally
model2 = keras.Sequential(name = "model_2")
model2.add(keras.Input(shape=(4,), name="input")) # specifying input shape
model2.add(layers.Dense(2, activation='relu', name="input_layer"))
model2.add(layers.Dense(3, activation='relu', name="hidden_dense_1"))
model2.add(layers.Dense(4, name="output_layer"))

## Model summary

You could inspect the architecture, the number of weights/biases, the shape of output at each layer etc. with `model.summary()`. 

Note: this is only possible if the input shape is specified for the model, i.e. only possible for option 2 above.

In [11]:
model2.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (Dense)          (None, 2)                 10        
_________________________________________________________________
hidden_dense_1 (Dense)       (None, 3)                 9         
_________________________________________________________________
output_layer (Dense)         (None, 4)                 16        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


This is extremely useful in debugging and also figuring out what layers to add for the model before training the model.

## Accessing layers

You could access layers to extract features from the layers, for example the output of a hidden layer. You could essentially create an additional model that extracts the outputs of all intermediate layers in a Sequential model.

This works because once a Sequential model has been built, every layer has an `input` and `output` attribute.

In [12]:
model2.layers # access all layers

In [13]:
model2.get_layer(name="hidden_dense_1") # access layer by name

In [14]:
# Creating a feature extractor model
feature_extractor = keras.Model(
    inputs = model2.inputs,
    outputs = [layer.output for layer in model2.layers]
)

You can call this feature extractor on inputs just like you would for a Sequential model.

For eg:

In [16]:
x = tf.ones((1, 4))
features = feature_extractor(x)

# What's next?
What do you do then with you model? You train, evaluate it, and use it to predict.

This will be covered in another notebook.

# Appendix

## A1. Layers
There are many pre-made layers available from `keras`.

Among those, the common ones include:
* `layers.Dense`
* `layers.Conv2D`
* `layers.MaxPooling2D`

etc.

Documentation for these layers could be found on the `keras` website.

### `layers.Dense()`
In this tutorial, `layers.Dense` was used. The mandatory argument it takes in is `units`, which specifies the shape of the output of the layer.

`Dense` implements the operation: `output = activation(dot(input, kernel) + bias)`. Hence, the weight kernel has a shape of `(input_dim, units)`.